## Preparing Images

k3d and kind are similar, they can't use local docker image, k3d need image in public or private registry server while kind need to use `load` command to import the image to it's internal CRI format.

### K3d

there are 2 options, one is use private registry k3d provides, the other one is use the command `k3d image import <image>`

host a private registry
```
k3d registry create registry.localhost --port 5000
```

**PS**, port 5000 may be occupied by airplay service in MacOS, can disable the service in `system preference / share / airplay`


when creating cluster, need to specify the extra parameter.
```
k3d cluster create --registry-use k3d-registry.localhost:5000
```

to upload the image
```
docker tag <image>:<tag> k3d-registry.localhost:5000/<image>:<tag>
docker push k3d-registry.localhost:5000/<image>:<tag>
```


In [7]:
!docker tag ml-zoomcamp_inference-gateway:latest k3d-registry.localhost:5000/ml-zoomcamp_inference-gateway:latest
!docker tag ml-zoomcamp_inference-ch2:latest k3d-registry.localhost:5000/ml-zoomcamp_inference-ch2:latest

### Kind

```
kind load docker-image ml-zoomcamp_inference-ch2:latest
kind load docker-image ml-zoomcamp_inference-gateway
```

List CRI images available for k8s.

In [3]:
!docker exec -it kind-control-plane crictl image

IMAGE                                             TAG                  IMAGE ID            SIZE
docker.io/kindest/kindnetd                        v20220510-4929dd75   31d1852b05abc       43.8MB
docker.io/kindest/local-path-helper               v20220512-507ff70b   9bc13867d19ca       2.75MB
docker.io/kindest/local-path-provisioner          v0.0.22-kind.0       7902f9a1c54fa       15.6MB
docker.io/library/ml-zoomcamp_inference-ch2       latest               1d5b92ee3172b       1GB
docker.io/library/ml-zoomcamp_inference-gateway   latest               1ff49d080c415       2.32GB
k8s.gcr.io/coredns/coredns                        v1.8.6               edaa71f2aee88       12.3MB
k8s.gcr.io/etcd                                   3.5.3-0              a9a710bb96df0       81.1MB
k8s.gcr.io/kube-apiserver                         v1.24.0              acde05a590759       75.5MB
k8s.gcr.io/kube-controller-manager                v1.24.0              b1d312727171f       63.8MB
k8s.gcr.io/kube-proxy    

## Create Cluster

Using k3d instead of kind just because it has bundled load balancer and ingress services already.

In [ ]:
!k3d cluster create ml-zoomcamp -p '9090:80@loadbalancer' --registry-use k3d-registry.localhost:5000

## Volume

doesn't use because yaml config doesn't support relative path.

# Network access

- internal
    through local dns
        - `<service_name>.<namespace>`, e.g gateway.default
        - `<service_name>.<namespace>.svc.cluster.local

- external
    - service, LoadBalancer, NodePort
        - NodePort doesn't work in mac, can't access the internal node ip.
    - port forward
        kubectl port-forward services/gateway 9999:8000
    - ingress

### Ingress with traefik

check [ingress.yaml](./ingress.yaml)

one entry to serve all backend service, there are 2 ways to differenciate services
- host name, with this, I have to use `/etc/hosts` file or config a local dns for the private domain
- prefix, this need traefik middle ware to stript the prefix or we'll get 404 response cause beckend service doesn't know the prefix path.

## Common commands

- `get`, return list
- `describe`, show detail

In [5]:
!kubectl get service -o wide

NAME         TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE   SELECTOR
kubernetes   ClusterIP   10.43.0.1    <none>        443/TCP   10m   <none>


In [6]:
!kubectl get deployment -o wide

No resources found in default namespace.


In [4]:
!kubectl get pods -o wide

No resources found in default namespace.
